In [23]:
!pip install spacy transformers spacy-transformers

In [2]:
import spacy
import en_core_med7_trf
import easyocr
from PIL import Image
import cv2
import pandas as pd
import csv
import re
import pyttsx3
medication_names = []

Text Detection from images

In [4]:
def detection():
    reader = easyocr.Reader(['en'])
    image_path = 'D:\\Programming-1\\New-Project\\Adobe Scan Mar 03, 2024_1.jpg'
    final_image = image_processing(image_path=image_path)
    result = reader.readtext(final_image)
    txt = []
    for detection in result:
        # print(detection[1])
        txt.append(detection[1])

    return " ".join(txt)


def image_processing(image_path):
    img = cv2.imread(image_path)
    reizised_img = img[200:1500, :]
    return reizised_img

Detection of text from given image

In [5]:
def med7_model():
  med7 = en_core_med7_trf.load()
  col_dict = {}
  seven_colours = ['#e6194B', '#3cb44b', '#ffe119', '#ffd8b1', '#f58231', '#f032e6', '#42d4f4']
  for label, color in zip(med7.pipe_labels['ner'], seven_colours):
    col_dict[label] = color

  options = {'ents': med7.pipe_labels['ner'], 'colors': col_dict}

  # Text containing medical concepts
  text = detection()  

  doc = med7(text)

  drug_data = []
  for ent in doc.ents:
    if ent.label_ == "DRUG":
      medication_names.append(ent.text)
      drug_data.append([ent.text, extract_frequency(ent.text, text)])  # Add frequency

  
  with open("drug_info_.csv", "w", newline="") as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(["Drug Name", "Frequency"])
    writer.writerows(drug_data)

  spacy.displacy.render(doc, style='ent', jupyter=True, options=options)

def extract_frequency(drug_name, text):
  frequency_pattern = re.compile(f"{re.escape(drug_name)} \((.*?)\)")

  match = re.search(frequency_pattern, text)
  if match:
    return match.group(1)  
  else:
    return None  

med7_model()


Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.


Assitant

In [2]:
import transformers
import torch
import google.generativeai as genai
from transformers import DPRContextEncoder, DPRContextEncoderTokenizer, DPRQuestionEncoder, DPRQuestionEncoderTokenizer
from transformers import AutoModelForCausalLM, AutoTokenizer

In [7]:
def speak(text):
    voice_id = 1
    engine = pyttsx3.init()
    voices = engine.getProperty('voices')
    engine.setProperty('voice', voices[voice_id].id)
    engine.setProperty('rate', 190) 
    engine.setProperty('volume', 0.9)
    engine.say(text)
    engine.runAndWait()

In [8]:
retriever_model_name = "facebook/dpr-ctx_encoder-single-nq-base"
retriever_tokenizer = DPRQuestionEncoderTokenizer.from_pretrained(retriever_model_name)
retriever_model = DPRContextEncoder.from_pretrained(retriever_model_name)

generator_model_name = "microsoft/DialoGPT-medium"
generator_tokenizer = AutoTokenizer.from_pretrained(generator_model_name)
generator_model = AutoModelForCausalLM.from_pretrained(generator_model_name)

In [1]:
def ask_about_drug(user_question):
    drug_info = {}
    with open("drug_info.csv", "r") as file:
        reader = csv.reader(file)
        next(reader)
        for row in reader:
            drug_name, drug_frequency = row
            drug_info[drug_name.lower()] = drug_frequency

    print(drug_info) # check 1
    # user_question = "How many times do i have to take salbair ?"

    query_tokens = user_question.lower().split()
    print(query_tokens)
    drug_name = None
    for token in query_tokens:
        if token in drug_info:
            drug_name = token
            break
    if drug_name:
        passage = f"{drug_name.capitalize()} is taken {drug_info[drug_name]}"
    else:
        try:
            genai.configure(api_key='APIKEY') 
            model = genai.GenerativeModel('gemini-pro')
            response = model.generate_content(user_question)
            passage = response.text
        except Exception as e:
            passage = f"Error accessing Gemini API: {e}"

    input_ids = generator_tokenizer.encode(user_question + " " + passage, return_tensors="pt")
    output = generator_model.generate(input_ids, max_new_tokens=100, num_return_sequences=1, pad_token_id=generator_tokenizer.eos_token_id)
    response = generator_tokenizer.decode(output[0], skip_special_tokens=True)

    print("Response:", response)
    speak(response)

ask_about_drug("Are there any side effects of salbair ?")


NameError: name 'csv' is not defined

In [12]:
# import csv

# def ask_about_drug():
#   drug_info = {}
#   with open("drug_info.csv", "r") as file:
#     reader = csv.reader(file)
#     next(reader)
#     for row in reader:
#       drug_name, drug_frequency = row
#       drug_info[drug_name.lower()] = drug_frequency

#   while True:
#     user_question = input("What would you like to know about a medication? (or type 'quit' to exit): ")
#     user_question = user_question.lower()

#     if user_question == 'quit':
#       break

#     query_tokens = user_question.split()
#     drug_name = None
#     for token in query_tokens:
#       if token in drug_info:
#         drug_name = token
#         break

#     if drug_name:
#       if any(word in user_question.lower() for word in ['frequency', 'times', 'take']):
#         passage = f"{drug_name.capitalize()} is taken {drug_info[drug_name]} times per day."  
#         print(passage)
#       else:
#         try:
#           genai.configure(api_key='APIKEY')
#           model = genai.GenerativeModel('gemini-pro')
#           response = model.generate_content(user_question)
#           passage = response.text
#           print(passage)
#         except Exception as e:
#           print(f"Error accessing Gemini API: {e}")
#     else:
#       print(f"Sorry, I don't have information about '{drug_name}'.")

# if __name__ == "__main__":
#   ask_about_drug()

import csv

def ask_about_drug():
  drug_info = {}
  with open("drug_info.csv", "r") as file:
    reader = csv.reader(file)
    next(reader)
    for row in reader:
      drug_name, drug_frequency = row
      drug_info[drug_name.lower()] = drug_frequency

  while True:
    user_question = input("What would you like to know about a medication? (or type 'quit' to exit): ")
    user_question = user_question.lower()

    if user_question == 'quit':
      break

    query_tokens = user_question.split()
    drug_name = None
    for token in query_tokens:
      if token in drug_info:
        drug_name = token
        break

    if drug_name:
      if any(word in user_question.lower() for word in ['frequency', 'times', 'take']):
        passage = f"{drug_name.capitalize()} is taken {drug_info[drug_name]} times per day."  
        print(passage)
      else:
        try:
          genai.configure(api_key='APIKEY')
          model = genai.GenerativeModel('gemini-pro')
          response = model.generate_content(user_question)
          passage = response.text
          passage = response.text.replace("*","")
          print(passage)
        except Exception as e:
          print(f"Error accessing Gemini API: {e}")
    else:
      print(f"Sorry, I don't have information about '{drug_name}'.")

if __name__ == "__main__":
  ask_about_drug()

Salbair is taken Thrice a Day times per day.
Sorry, I don't have information about 'None'.


Final Input to the Model

In [9]:
import speech_recognition as sr
from fuzzywuzzy import process
import pyttsx3


In [10]:
def input_speech():
    while True:
        print(f"\nListening .....")
        sp = sr.Recognizer()
        with sr.Microphone() as source:
            sp.pause_threshold = 0.5
            sp.energy_threshold = 3500
            audio = sp.listen(source)
            txt = sp.recognize_google(audio, language="en-in")
            print(f"User says {txt}")
            new_txt = autocorrect_medication(txt)
            print(new_txt)
            ask_about_drug(new_txt)
            print("Stoppped")
            return txt
        


def autocorrect_medication(input_text):
    # Split input into individual words
    words = input_text.split()
    corrected_words = []
    for word in words:
        closest_match = process.extractOne(word, medication_names)
        if closest_match is not None:
            matched_word, similarity = closest_match
            if similarity >= 70:  
                corrected_words.append(matched_word)
            else:
                corrected_words.append(word)  
        else:
            corrected_words.append(word) 
    return ' '.join(corrected_words)

In [12]:
input_speech()


Listening .....
User says what is the frequency of zero stand
what is the frequency of Zerostat Zerostat
{'zerostat': 'Twice a day', 'momeflo': 'Twice a Day', 'salbair': 'Thrice a Day', 'levocet': 'Once a Day'}
['what', 'is', 'the', 'frequency', 'of', 'zerostat', 'zerostat']
Response: what is the frequency of Zerostat Zerostat Zerostat is taken Twice a day.
Stoppped


'what is the frequency of zero stand'